In [120]:
import json as j
import pandas as pd

import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression


data=pd.read_csv('C:\\Users\\Yassine\\Desktop\\Étude\\3395\\competition\\train.csv', sep=',',header=0).to_numpy()
X_train = data[:,1]
y_train = data[:,2]
data_test =pd.read_csv('C:\\Users\\Yassine\\Desktop\\Étude\\3395\\competition\\test.csv', sep=',',header=0).to_numpy()

X_test = data_test[:,1] 

all_data = np.append(X_train,X_test)



In [121]:
no_word = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he'
           , 'him', 'his', 'himself', 'she', 'her','hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs'
           ,'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am','is', 'are', 'was', 'were'
           , 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and'
           , 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against'
           , 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in'
           , 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why'
           , 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only',
           'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 'd', 'll', 'm'
           , 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 
           'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won', 'wouldn']

greek= ["alpha", "beta", "gamma", "delta", "epsilon", "zata", "eta", "theta", "iota","kappa", "lambda", "mu", "nu", "xi", "omikron", "pi", "rho", "sigma", "tau", "upsilon", "phi","chi", "psi", "omega"]

documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(all_data)):
    # Remove all the special characters
    document = str(all_data[sen]).replace('\n',' ')
    document = re.sub(r'\W', ' ',document )
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    '''document = document.lower()
    for i in astro_words:
        if i in document:
            document = document.replace(i,(i.replace(' ','')+' '))'''
    # Lemmatization
    document = document.split()
    for i in range(len(document)):
        for m in greek:
            if m in document[i]:
                document[i] = m
    document = [stemmer.lemmatize(word) for word in document if (word.lower() not in no_word) and (not word.isdigit()) and(word.lower() not in stopwords.words('english')) ]
    document = ' '.join(document)
    
    documents.append(document)

In [132]:
victor = TfidfVectorizer(sublinear_tf=True, min_df=3, norm='l2', encoding='latin-1', ngram_range=(1, 2), lowercase=False)
r = victor.fit_transform(documents).toarray()

In [133]:
X_train_t = r[:len(X_train)]
X_test_t = r[len(X_train):]

In [134]:
from sklearn.naive_bayes import MultinomialNB

g2=MultinomialNB(alpha=0.15)
g2.fit(X_train_t,y_train)

MultinomialNB(alpha=0.15)

In [135]:
from sklearn.naive_bayes import MultinomialNB

g1=LogisticRegression()
g1.fit(X_train_t,y_train)

LogisticRegression()

In [136]:
g = g1.predict_proba(X_train_t)+g2.predict_proba(X_train_t)

In [137]:
gf=MultinomialNB()
gf.fit(g,y_train)

MultinomialNB()

In [138]:
g = g1.predict_proba(X_test_t)+g2.predict_proba(X_test_t)

In [139]:
pred = gf.predict(g)

In [140]:
s = pd.DataFrame(np.array(pred)).to_csv("C:/Users/Yassine/Desktop/Étude/3395/competition/predictions_9.csv")


In [68]:
g = g1.predict_proba(X_test_t)+g2.predict_proba(X_test_t)